In [2]:
def next_secret(current):
    MODULO = 16777216
    # Step 1: Multiply by 64
    step1 = (current * 64) % MODULO
    current = current ^ step1

    # Step 2: Integer division by 32
    step2 = current // 32
    current = current ^ step2
    
    # Step 3: Multiply by 2048
    step3 = (current * 2048) % MODULO
    current = current ^ step3
    
    return current % MODULO

def simulate_secrets_and_prices(initial_secret):
    iterations = 2000
    secrets = [initial_secret]
    for _ in range(iterations):
        new_secret = next_secret(secrets[-1])
        secrets.append(new_secret)
    prices = [secret % 10 for secret in secrets]
    return prices

def calculate_price_changes(prices):
    return [prices[i] - prices[i-1] for i in range(1, len(prices))]

def find_most_profitable_sequence(all_changes, all_prices):
    from collections import defaultdict
    change_to_prices = defaultdict(list)
    
    # Collect all possible sequences of four changes and corresponding prices
    for changes, prices in zip(all_changes, all_prices):
        for i in range(len(changes) - 3):
            sequence = tuple(changes[i:i+4])
            price = prices[i+5]  # Price after the sequence
            change_to_prices[sequence].append(price)
    
    # Determine the sequence that maximizes the total bananas
    best_sequence = max(change_to_prices, key=lambda seq: sum(change_to_prices[seq]))
    best_bananas = sum(change_to_prices[best_sequence])
    
    return best_sequence, best_bananas

def main(initial_secrets):
    all_changes = []
    all_prices = []
    for secret in initial_secrets:
        prices = simulate_secrets_and_prices(secret)
        changes = calculate_price_changes(prices)
        all_changes.append(changes)
        all_prices.append(prices)
    
    best_sequence, max_bananas = find_most_profitable_sequence(all_changes, all_prices)
    return best_sequence, max_bananas

# Example initial secrets
initial_secrets = [1, 2, 3, 2024]

# Solving the problem
best_sequence, max_bananas = main(initial_secrets)
print(f"The best sequence of price changes is: {best_sequence}")
print(f"The maximum number of bananas obtained is: {max_bananas}")


IndexError: list index out of range

In [3]:
def find_most_profitable_sequence(all_changes, all_prices):
    from collections import defaultdict
    change_to_prices = defaultdict(list)
    
    # Collect all possible sequences of four changes and corresponding prices
    for changes, prices in zip(all_changes, all_prices):
        for i in range(len(changes) - 3):
            sequence = tuple(changes[i:i+4])
            # Access the price after the sequence correctly
            if i + 4 < len(prices):
                price = prices[i+4]  # Price corresponding to the end of the sequence
                change_to_prices[sequence].append(price)
    
    # Determine the sequence that maximizes the total bananas
    best_sequence = max(change_to_prices, key=lambda seq: sum(change_to_prices[seq]) if change_to_prices[seq] else 0)
    best_bananas = sum(change_to_prices[best_sequence]) if best_sequence in change_to_prices else 0
    
    return best_sequence, best_bananas

def main(initial_secrets):
    all_changes = []
    all_prices = []
    for secret in initial_secrets:
        prices = simulate_secrets_and_prices(secret)
        changes = calculate_price_changes(prices)
        all_changes.append(changes)
        all_prices.append(prices)
    
    best_sequence, max_bananas = find_most_profitable_sequence(all_changes, all_prices)
    return best_sequence, max_bananas

# Example initial secrets
initial_secrets = [1, 2, 3, 2024]

# Solving the problem
best_sequence, max_bananas = main(initial_secrets)
print(f"The best sequence of price changes is: {best_sequence}")
print(f"The maximum number of bananas obtained is: {max_bananas}")


The best sequence of price changes is: (-2, 2, -1, -1)
The maximum number of bananas obtained is: 30


In [4]:
def generate_next_secret(secret_number):
    secret_number ^= (secret_number * 64) % 16777216
    secret_number %= 16777216
    secret_number ^= (secret_number // 32) % 16777216
    secret_number %= 16777216
    secret_number ^= (secret_number * 2048) % 16777216
    secret_number %= 16777216
    return secret_number

def get_price_sequence(initial_secret):
    # Generate all prices at once
    prices = []
    secret = initial_secret
    for _ in range(2001):  # We need 2001 to get 2000 changes
        prices.append(secret % 10)
        secret = generate_next_secret(secret)
    return prices

def find_sequences(prices):
    # Create a dictionary to store where each sequence appears and its corresponding price
    sequences = {}
    changes = []
    
    # Calculate all changes
    for i in range(1, len(prices)):
        changes.append(prices[i] - prices[i-1])
    
    # For each position, record the 4-change sequence that starts there
    for i in range(len(changes) - 3):
        seq = tuple(changes[i:i+4])  # Convert to tuple so it can be used as dict key
        if seq not in sequences:  # Only keep the first occurrence
            sequences[seq] = prices[i+4]
            
    return sequences

def main():
    with open("input.txt", "r") as file:
        initial_secrets = [int(line.strip()) for line in file.readlines()]
    
    # Pre-calculate all sequences for each buyer
    buyer_sequences = []
    for secret in initial_secrets:
        prices = get_price_sequence(secret)
        sequences = find_sequences(prices)
        buyer_sequences.append(sequences)
    
    # Find the sequence that appears in most buyers with highest total
    best_total = 0
    best_sequence = None
    
    # Get all unique sequences that appear in any buyer's data
    all_sequences = set()
    for sequences in buyer_sequences:
        all_sequences.update(sequences.keys())
    
    # Check each sequence that actually appears in the data
    for seq in all_sequences:
        total = sum(sequences.get(seq, 0) for sequences in buyer_sequences)
        if total > best_total:
            best_total = total
            best_sequence = seq
    
    print(f"Best sequence: {list(best_sequence)}")
    print(f"Maximum bananas: {best_total}")

if __name__ == "__main__":
    main()

Best sequence: [1, -3, 3, 1]
Maximum bananas: 2152
